In [12]:
import sqlite3
import pandas as pd
DB_PATH = "data/predictions.db"

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

# List all tables
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

# Check columns in the predictions table
cur.execute("PRAGMA table_info(predictions);")
columns = cur.fetchall()


df = pd.read_sql_query("SELECT * FROM predictions", conn)
conn.close()
df

,id,city,current_temp,current_dwpt,current_hum,current_prcp,current_wdir,current_wspd,current_wpgt,current_pres,current_coco,pred_temp,pred_hum,pred_pres,pred_wspd,actual_temp,actual_hum,actual_pres,actual_wspd,timestamp
0,1,Dubai,36.83,21.776546,42.0,0.0,282.0,3.73,3.73,1003.0,3.0,36.746876,42.922857,1003.034011,8.316164,NaN,NaN,NaN,NaN,2025-09-10T12:22:37
1,2,New York,16.74,10.577189,67.0,0.0,54.0,5.71,8.13,1024.0,3.0,16.959419,66.397152,1023.964226,9.054329,NaN,NaN,NaN,NaN,2025-09-10T12:28:00
2,3,Dubai,36.83,21.776546,42.0,0.0,282.0,3.73,3.73,1003.0,3.0,36.746876,42.922857,1003.034011,8.316164,NaN,NaN,NaN,NaN,2025-09-10T12:34:37
3,4,Mumbai,27.79,21.087330,67.0,0.0,236.0,3.04,2.63,1009.0,3.0,27.846643,66.830505,1009.124403,5.993499,NaN,NaN,NaN,NaN,2025-09-10T13:39:07
4,5,Mumbai,27.79,21.087330,67.0,0.0,236.0,3.04,2.63,1009.0,3.0,27.846643,66.830505,1009.124403,5.993499,NaN,NaN,NaN,NaN,2025-09-10T13:39:09
5,6,Mumbai,27.79,21.087330,67.0,0.0,236.0,3.04,2.63,1009.0,3.0,27.846643,66.830505,1009.124403,5.993499,27.79,67.0,1009.0,3.04,2025-09-10T13:40:11
6,7,Dubai,37.30,25.430832,51.0,0.0,330.0,6.03,6.08,1001.0,3.0,37.158257,51.474020,1001.099525,9.887034,NaN,NaN,NaN,NaN,2025-09-10T16:24:55
7,8,Tokyo,29.05,22.284369,67.0,0.0,185.0,7.36,9.10,1012.0,3.0,28.986974,67.064017,1011.981377,10.271309,29.19,65.0,1012.0,7.42,2025-09-10T16:28:41
8,9,Tokyo,29.19,21.918811,65.0,0.0,191.0,7.42,9.25,1012.0,3.0,29.079147,65.303814,1011.980240,10.328185,NaN,NaN,NaN,NaN,2025-09-10T17:29:13


In [10]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 52.52,
	"longitude": 13.41,
	"hourly": ["temperature_2m", "relative_humidity_2m"],
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

Coordinates: 52.52000045776367°N 13.419998168945312°E
Elevation: 38.0 m asl
Timezone difference to GMT+0: 0s

Hourly data
                          date  temperature_2m  relative_humidity_2m
0   2025-09-10 00:00:00+00:00       18.395500                  80.0
1   2025-09-10 01:00:00+00:00       18.295500                  81.0
2   2025-09-10 02:00:00+00:00       18.095499                  84.0
3   2025-09-10 03:00:00+00:00       17.395500                  83.0
4   2025-09-10 04:00:00+00:00       17.345499                  82.0
..                        ...             ...                   ...
163 2025-09-16 19:00:00+00:00       17.419500                  65.0
164 2025-09-16 20:00:00+00:00       16.519501                  67.0
165 2025-09-16 21:00:00+00:00       15.769500                  68.0
166 2025-09-16 22:00:00+00:00       15.119500                  69.0
167 2025-09-16 23:00:00+00:00       14.569500                  70.0

[168 rows x 3 columns]
